<h1> 1. Imports </h1>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
from torchvision import models
from torchvision.io import read_image
import torchmetrics

import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import PIL

In [2]:
def read_labels(root_dir: str, le: LabelEncoder):
    dict = {"filepath":[], "labels":[]}

    for root, _, files in os.walk(root_dir):
        for file in files:
            dict["filepath"].append(os.path.join(root, file))

            dict["labels"].append(root.split("\\")[-1])

    labels_df = pd.DataFrame(dict)
    labels_df['labels'] = le.fit_transform(labels_df['labels'])
    return labels_df

In [3]:
le = LabelEncoder()
labels_df = read_labels("data", le)

In [4]:
labels_df

,filepath,labels
0,data\Testing\glioma_tumor\image(1).jpg,0
1,data\Testing\glioma_tumor\image(10).jpg,0
2,data\Testing\glioma_tumor\image(100).jpg,0
3,data\Testing\glioma_tumor\image(11).jpg,0
4,data\Testing\glioma_tumor\image(12).jpg,0
...,...,...
3259,data\Training\pituitary_tumor\p (95).jpg,3
3260,data\Training\pituitary_tumor\p (96).jpg,3
3261,data\Training\pituitary_tumor\p (97).jpg,3
3262,data\Training\pituitary_tumor\p (98).jpg,3


In [5]:
class BrainTumorDataset(Dataset):
    def __init__(self, root_dir, labels_df, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.labels_df = labels_df

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # image = read_image(self.labels_df.iloc[idx, 0])
        image = PIL.Image.open(self.labels_df.iloc[idx, 0])
        label = self.labels_df.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)
        
        return image, label

#### Custom Net

In [26]:
class VGG_BrainTumorNet(nn.Module):
    def __init__(self, input_channels, output_classes, architecture="VGG16"):
        super(VGG_BrainTumorNet, self).__init__()

        VGG = [64, 64, "p", 128, 128, "p", 256, 256, 256, "p", 512, 512, 512, "p", 512, 512, 512, "p"]

        self.input_channels = input_channels
        self.output_classes = output_classes

        self.model = self._build_architecture(VGG)
        

    def forward(self, x):
        return self.model(x)

    def _build_architecture(self, architecture):
        layers = []
        count_max_pool_layers = 0
        in_channels = self.input_channels

        # adding convolutional layers
        for x in architecture:
            if type(x) == int:
                out_channels = x

                layers += [
                    nn.Conv2d(in_channels, out_channels, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.BatchNorm2d(out_channels),
                    nn.ReLU()
                    ]
                
                in_channels = x
            else:
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]
                count_max_pool_layers += 1


        conv_output_channels = (512 // (2**count_max_pool_layers))**2

        # adding fully connected layers
        layers += [
            nn.Flatten(),
            nn.Linear(512*conv_output_channels, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, self.output_classes)
        ]

        layers = nn.ModuleList(layers)

        return nn.Sequential(*layers)

#### Transfer Learning

In [7]:
m = models.vit_b_32(pretrained=True)

In [8]:
print(m)

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (linear_1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU()
          (dropout_1): Dropout(p=0.0, inplace=False)
          (linear_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout_2): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
 

In [9]:
class ViT_BrainTumor(nn.Module):
    def __init__(self, image_channels, output_classes):
        super(ViT_BrainTumor, self).__init__()

        self.image_channels = image_channels
        self.output_classes = output_classes

        model = models.vit_b_16(pretrained=True)

        self.conv_proj = model.conv_proj
        self.encoder = model.encoder
        self.heads = nn.Sequential(
            nn.Linear(768, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, self.output_classes)
        )

        self._freeze_weights(self.conv_proj)
        self._freeze_weights(self.encoder)

    
    def _freeze_weights(self, layers):
        for param in layers.parameters():
            param.requires_grad = False

    def forward(self, x):
        x1 = self.conv_proj(x)
        x2 = self.encoder(x1)
        x3 = self.heads(x2)

        return x3

#### Training

In [10]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize((512, 512)),
        transforms.Normalize(mean=(0.5,), std=(0.5,))
    ]
)

In [11]:
dataset_train = BrainTumorDataset("data/Training", labels_df, transform=transform)

dataset_test = BrainTumorDataset("data/Testing", labels_df, transform=transform)

In [12]:
loader_train = DataLoader(dataset_train, batch_size=64, shuffle=True)

loader_test = DataLoader(dataset_test, batch_size=64, shuffle=True)

In [13]:
def save_model(filepath, epoch, model_state_dict, optimizer_state_dict, loss):
    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": model_state_dict,
            "optimizer_state_dict": optimizer_state_dict,
            "loss": loss
        },
        filepath
    )

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [33]:
def training_loop(
    model: nn.Module, 
    device,
    loader_train: DataLoader, 
    loader_test: DataLoader, 
    epochs: int, 
    learning_rate: float, 
    weight_decay: float, 
    verbose: bool = True, 
    filepath: str = "model"
    ):

    

    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    loss_function = nn.CrossEntropyLoss()

    for epoch in range(epochs):

        training_loss=0
        validation_loss=0

        model.train()

        for idx, (features, labels) in enumerate(loader_train):
            features = features.to(device)
            labels = labels.to(device)

            predicted = model(features)

            loss = loss_function(predicted, labels)
            training_loss+=loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        model.eval()

        with torch.no_grad():
            for idx, (features, labels) in enumerate(loader_test):

                features = features.to(device)
                labels = labels.to(device)

                predicted = model(features)

                loss = loss_function(predicted, labels)
                validation_loss+=loss

        if verbose:
            print(f"On epoch: {epoch}, Training loss: {training_loss}, Validation loss: {validation_loss}")

        if epoch % 10 == 0:
            save_model(filepath, epoch, model.state_dict(), optimizer.state_dict(), training_loss)

In [22]:
epochs = 30
learning_rate = 3e-4
weight_decay = 0.01
filepath = "model/vgg"

In [27]:
vgg = VGG_BrainTumorNet(3, 4).to(device)

In [34]:
training_loop(vgg, device, loader_train, loader_test, epochs, learning_rate, weight_decay, verbose=True, filepath=filepath)

RuntimeError: CUDA out of memory. Tried to allocate 4.00 GiB (GPU 0; 4.00 GiB total capacity; 2.31 GiB already allocated; 957.70 MiB free; 2.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF